In [2]:
import numpy as np
from cobra import Model, Reaction, Metabolite
import sys
import os

In [12]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

#### Use the toy model in Ben's paper

In [5]:
m1 = Metabolite('M1',compartment='c')
m2 = Metabolite('M2',compartment='c')
m3 = Metabolite('M3',compartment='c')
m4 = Metabolite('M4',compartment='c')

In [6]:
r1 = Reaction('R1',name='Reaction1')
r1.add_metabolites({m1:1})
r1.lower_bound = 0
r1.upper_bound = 1000

r2 = Reaction('R2',name='Reaction2')
r2.add_metabolites({m2:1})
r2.lower_bound = 0
r2.upper_bound = 1000

r3 = Reaction('R3',name='Reaction3')
r3.add_metabolites({m1:-1,m2:-1,m3:1})
r3.gene_reaction_rule = 'E1'
r3.lower_bound = 0
r3.upper_bound = 1000

r4 = Reaction('R4',name='Reaction4')
r4.add_metabolites({m3:-1,m4:1})
r4.gene_reaction_rule = 'E1 or E2'
r4.lower_bound = 0
r4.upper_bound = 1000

r5 = Reaction('R5',name='Reaction5')
r5.add_metabolites({m2:-1,m4:1})
r5.gene_reaction_rule = 'E3 and E4'
r5.lower_bound = -1000
r5.upper_bound = 1000

r6 = Reaction('R6',name='Reaction6')
r6.add_metabolites({m4:-1})
r6.lower_bound = 0
r6.upper_bound = 1000

In [7]:
model = Model()
model.add_reactions([r1,r2,r3,r4,r5,r6])

Using license file /Users/gangl/gurobi.lic
Academic license - for non-commercial use only


In [11]:
utils.report_model_rxns(model)

R1 Reaction1 
 --> M1 

R2 Reaction2 
 --> M2 

R3 Reaction3 E1
M1 + M2 --> M3 

R4 Reaction4 E1 or E2
M3 --> M4 

R5 Reaction5 E3 and E4
M2 <=> M4 

R6 Reaction6 
M4 -->  



#### Convert to irreversible model 

In [13]:
irrModel = ecpy.convertToIrrev(model)

In [14]:
utils.report_model_rxns(irrModel)

R1 Reaction1 
 --> M1 

R2 Reaction2 
 --> M2 

R3 Reaction3 E1
M1 + M2 --> M3 

R4 Reaction4 E1 or E2
M3 --> M4 

R5 Reaction5 E3 and E4
M2 --> M4 

R5_REV Reaction5 (reversible) E3 and E4
M4 --> M2 

R6 Reaction6 
M4 -->  



#### convert to ecmodel

In [15]:
kcats = {
     ('E1','R3'): 10,
     ('E1','R4'): 100,
     ('E2','R4'): 0.10,
     ('E3','R5'): 5,
     ('E3','R5_REV'): 12,
     ('E4','R5'): 15,
     ('E4','R5_REV'): 90,
       }
MWs = {
    'E1': 1000,
    'E2': 3000,
    'E3': 5000,
    'E4': 6000,
    'E5': 10000,
}

In [16]:
eModel = ecpy.convertToEnzymeModel(irrModel,kcats)

In [17]:
utils.report_model_rxns(eModel)

R1 Reaction1 
 --> M1 

R2 Reaction2 
 --> M2 

R3No1 Reaction3 (No1) E1
M1 + M2 + 0.1 prot_E1_c --> M3 

arm_R4 Reaction4 (arm) 
M3 --> pmet_R4 

R4No1 Reaction4 (No1) E1
pmet_R4 + 0.01 prot_E1_c --> M4 

R4No2 Reaction4 (No2) E2
pmet_R4 + 10.0 prot_E2_c --> M4 

R5No1 Reaction5 (No1) E3 and E4
M2 + 0.2 prot_E3_c + 0.2 prot_E4_c --> M4 

R5_REVNo1 Reaction5 (reversible) (No1) E3 and E4
M4 + 0.08333333333333333 prot_E3_c + 0.08333333333333333 prot_E4_c --> M2 

R6 Reaction6 
M4 -->  

prot_E1_exchange  E1
 --> prot_E1_c 

prot_E2_exchange  E2
 --> prot_E2_c 

prot_E3_exchange  E3
 --> prot_E3_c 

prot_E4_exchange  E4
 --> prot_E4_c 



#### add prot_pool  

In [18]:
ecModel = ecpy.constrainPool(eModel,MWs, ['E1','E2','E4'],10)

Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpgged3_xz.lp
Reading time = 0.01 seconds
: 9 rows, 26 columns, 54 nonzeros


In [19]:
utils.report_model_rxns(ecModel)

R1 Reaction1 
 --> M1 

R2 Reaction2 
 --> M2 

R3No1 Reaction3 (No1) E1
M1 + M2 + 0.1 prot_E1_c --> M3 

arm_R4 Reaction4 (arm) 
M3 --> pmet_R4 

R4No1 Reaction4 (No1) E1
pmet_R4 + 0.01 prot_E1_c --> M4 

R4No2 Reaction4 (No2) E2
pmet_R4 + 10.0 prot_E2_c --> M4 

R5No1 Reaction5 (No1) E3 and E4
M2 + 0.2 prot_E3_c + 0.2 prot_E4_c --> M4 

R5_REVNo1 Reaction5 (reversible) (No1) E3 and E4
M4 + 0.08333333333333333 prot_E3_c + 0.08333333333333333 prot_E4_c --> M2 

R6 Reaction6 
M4 -->  

prot_E3_exchange  E3
 --> prot_E3_c 

draw_prot_E1 draw_prot_E1 
1000 prot_pool --> prot_E1_c 

draw_prot_E2 draw_prot_E2 
3000 prot_pool --> prot_E2_c 

draw_prot_E4 draw_prot_E4 
6000 prot_pool --> prot_E4_c 

prot_pool_exchange prot_pool_exchange 
 --> prot_pool 



In [20]:
ecModel.objective = 'R6'

In [21]:
s = ecModel.optimize()

In [22]:
print(s.objective_value)
s.fluxes

0.09090909090909091


R1                     0.090909
R2                     0.090909
R3No1                  0.090909
arm_R4                 0.090909
R4No1                  0.090909
R4No2                  0.000000
R5No1                  0.000000
R5_REVNo1              0.000000
R6                     0.090909
prot_E3_exchange       0.000000
draw_prot_E1           0.010000
draw_prot_E2           0.000000
draw_prot_E4           0.000000
prot_pool_exchange    10.000000
Name: fluxes, dtype: float64

#### Add proteomics data

In [23]:
enzyme_abandance = {'E3':0.001}

In [24]:
for enz, ab in enzyme_abandance.items():
    rxn = ecModel.reactions.get_by_id('prot_{0}_exchange'.format(enz))
    rxn.lower_bound = ab

In [25]:
s = ecModel.optimize()
print(s.objective_value)
s.fluxes

0.04136363636363636


R1                     0.036364
R2                     0.041364
R3No1                  0.036364
arm_R4                 0.036364
R4No1                  0.036364
R4No2                  0.000000
R5No1                  0.005000
R5_REVNo1              0.000000
R6                     0.041364
prot_E3_exchange       0.001000
draw_prot_E1           0.004000
draw_prot_E2           0.000000
draw_prot_E4           0.001000
prot_pool_exchange    10.000000
Name: fluxes, dtype: float64